In [6]:
import torch
import torch.nn as nn
from torchtext.datasets import PennTreebank
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset, DataLoader

In [4]:
tokenizer = get_tokenizer('basic_english')
def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(PennTreebank(split='train')), 
                                  specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])
tokens = [token for line in PennTreebank(split='train') for token in tokenizer(line)]
indexed_data = [vocab[token] for token in tokens]

NameError: name 'get_tokenizer' is not defined

In [ ]:
def create_batches(data, seq_len=30, batch_size=32):
    total_len = (len(data) // (batch_size * seq_len)) *  batch_size * seq_len
    data = torch.tensor(data[:total_len], dtype=torch.long)
    data = data.view(batch_size, -1)

    X, Y = [], []
    for i in range(0, data.size(1) - seq_len):
        x = data[:, i:i+seq_len]
        y = data[:, i+1:i+seq_len+1]
        x.append(x)
        y.append(y)
    
    return torch.stack(X), torch.stack(Y)